In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [5]:
# add the project root to the path
import sys
sys.path.append("/orcd/data/omarabu/001/njwfish/counting_flows")

In [7]:
import hydra
from hydra.core.global_hydra import GlobalHydra

GlobalHydra.instance().clear()
# setup hydra config global for loading this notebook
hydra.initialize(config_path="../../configs", version_base=None)
cfg = hydra.compose(config_name="expr")


In [ ]:
from main import setup_environment, get_checkpoint_info, is_training_complete

# Setup environment
device = setup_environment(cfg)

# Get checkpoint info
output_dir, checkpoint = get_checkpoint_info(cfg, num_epochs=cfg.training.num_epochs)

# Instantiate everything
bridge = hydra.utils.instantiate(cfg.bridge)
dataset = hydra.utils.instantiate(cfg.dataset)

18:48:01 - INFO - Using device: cuda
18:48:03 - INFO - Using compile: False


Train split: 931, Test split: 49
Train cells: 1202750, Test cells: 61517


/home/njwfish/miniconda3/envs/seq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: 'NoneType' object is not subscriptable

In [17]:
output_dir, checkpoint = get_checkpoint_info(cfg, num_epochs=cfg.training.num_epochs)

/orcd/data/omarabu/001/njwfish/counting_flows


18:54:43 - INFO - Found checkpoint: /orcd/data/omarabu/001/njwfish/counting_flows/outputs/84cddc73ed43/model.pt


In [18]:
model = hydra.utils.instantiate(cfg.model)
model.load_state_dict(checkpoint['model_state_dict'])
avg_model = hydra.utils.instantiate(cfg.averaging, model=model)
avg_model.load_state_dict(checkpoint['avg_model_state_dict'])
    

<All keys matched successfully>

In [22]:
import numpy as np
import torch

ctx_window_size    = 196_608
target_window_size = 896
stride             = 896   # tiles exactly; use <= 896 to allow overlap

whalf = ctx_window_size // 2
thalf = target_window_size // 2

base_idx = dataset.base_individual_idx  # (n_cells,)
target_idxs = dataset.test_individual_idxs

device = "cuda"  # or "cpu"

for seed in range(1, 3):
    print(seed)
    gene_counts = {}
    gene_counts_true = {}
    for target_key in target_idxs:
        if target_key in gene_counts:
            continue
        gene_counts[target_key] = {}
        gene_counts_true[target_key] = {}
        target_idx = target_idxs[target_key]
        try:    
            for ci, gs, ge, gn in zip(dataset._eligible_cidx, dataset._eligible_start, dataset._eligible_end, dataset._eligible_gene_name):

                chrom_beg  = dataset.chrom_starts[ci]
                gene_start = int(chrom_beg + int(gs))
                gene_end   = int(chrom_beg + int(ge))  # end-exclusive
                gene_len   = int(gene_end - gene_start)

                print(gene_len)

                big_seq = dataset.get_seq(gene_start - whalf, gene_end + whalf)
                assert len(big_seq) == gene_len + 2*whalf

                # Centers whose 896bp target is fully inside the gene
                first_center = thalf
                last_center  = gene_len - thalf
                # if last_center < first_center:
                #     continue  # gene shorter than target; skip or handle separately

                centers_local = np.arange(first_center, last_center + 1, stride, dtype=np.int64)
                if len(centers_local) == 0:
                    centers_local = np.array([first_center])
                # ensure we always include a tail center that lands exactly on the end
                if centers_local[-1] != last_center and centers_local[-1] != first_center:
                    centers_local = np.append(centers_local, last_center)

                # Precompute per-cell coverage over the whole gene once; we’ll slice per window
                # (shape: [n_cells, gene_len])
                # gene_end_extended shuold be a multiple of 896
                gene_end_extended = gene_start + ((gene_len - 1) // 896 + 1) * 896
                print(target_key, gene_end - gene_start, gene_end_extended - gene_start, gene_end < gene_end_extended)
                x_1_global = torch.from_numpy(
                    dataset.fast_get_overlap_raw(base_idx[:target_idx.shape[0]], (gene_start, gene_end_extended))
                )# .to(device)

                gene_counts_true[target_key][gn] = dataset.fast_get_overlap_raw(target_idx, (gene_start, gene_end_extended))
                target_sum_global = torch.from_numpy(gene_counts_true[target_key][gn]).sum(dim=0)
                gene_counts_true[target_key][gn] = gene_counts_true[target_key][gn][:,:gene_len]

                # Output buffer: counts per cell along the gene
                count_arr = np.zeros((target_idx.shape[0], gene_len), dtype=np.int32)

                for c in centers_local:
                    # Context sequence for this center
                    seq = big_seq[c : c + ctx_window_size]
                    # Target genomic slice indices in gene-local coords
                    g0 = int(c - thalf)
                    g1 = g0 + target_window_size


                    # Slice the “conditioning” counts for these positions
                    x_1 = x_1_global[:, g0:g1].to(device)  # [n_cells, 896]
                    # Build context dict for your model
                    context = {
                        'seq': torch.from_numpy(seq).to(device),
                        'class_emb': torch.zeros(
                            target_idx.shape[0], dataset.target_cond.shape[1], device=device
                        ),
                        'target_sum': target_sum_global[g0:g1].unsqueeze(0).to(device),
                        'A': torch.ones(1, target_idx.shape[0], device=device).to(device)
                    }

                    with torch.no_grad():
                        target_counts = bridge.sampler(
                            x_1,
                            context,
                            (avg_model.module if avg_model is not None else model).to(device),
                            n_steps=3,
                        )  # expect [n_cells, 896]

                    # Write into the gene buffer
                    count_arr[:, g0:g1] = target_counts.to("cpu").numpy()[:, :gene_len]

                gene_counts[target_key][gn] = count_arr
                print("counts match",np.all(count_arr.sum(axis=0) == gene_counts_true[target_key][gn].sum(axis=0)))
        except Exception as e:
            import gc
            torch.cuda.empty_cache()
            gc.collect()
            continue
        # save gene_counts with pickle
        import pickle as pkl
        pkl.dump(gene_counts, open(f"/orcd/data/omarabu/001/njwfish/counting_flows/results/expr/deconv_gene_counts_proj_{seed}.pkl", "wb"))
        # pkl.dump(gene_counts_true, open("results/expr/deconv_gene_counts_true.pkl", "wb"))

1
523
548_549 523 896 True
counts match True
7098
548_549 7098 7168 True
counts match True
26277
548_549 26277 26880 True
counts match True
5077
548_549 5077 5376 True
counts match True
7105
548_549 7105 7168 True
counts match True
36280
548_549 36280 36736 True
counts match True
37494
548_549 37494 37632 True
counts match True
2396
548_549 2396 2688 True
counts match True
21619
548_549 21619 22400 True
counts match True
4522
548_549 4522 5376 True
counts match True
7814
548_549 7814 8064 True
counts match True
28181
548_549 28181 28672 True
counts match True
4671
548_549 4671 5376 True
counts match True
6729
548_549 6729 7168 True
counts match True
4514
548_549 4514 5376 True
counts match True
2784
548_549 2784 3584 True
counts match True
9089
548_549 9089 9856 True
counts match True
7508
548_549 7508 8064 True
counts match True
21570
548_549 21570 22400 True
counts match True
5127
548_549 5127 5376 True
counts match True
523
765_766 523 896 True
523
832_833 523 896 True
523
677_678 5